Fire polygons are retrieved from CAL FIRE historical records. This notebook demonstrates how to query the CAL FIRE open data interface for ignition coordinates and perimeter updates of the 2019 Maria Fire in Ventura County, CA.

In [4]:
import os
import sys
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
import geojson
import shapely
from shapely.geometry import Polygon, Point, mapping
import rasterio
from rasterio import plot, mask
from rasterio.plot import show
import zipfile
import time
import io
from PIL import Image
from PIL.TiffTags import TAGS
from osgeo import gdal

OUTDIR = "example_outputs"   # or keep your original name
os.makedirs(OUTDIR, exist_ok=True)

In [5]:
# MARIA 2019 historical perimeter updates
GEOMAC_URL = "https://services3.arcgis.com/T4QMspbfLg3qTGWY/arcgis/rest/services/Historic_GeoMAC_Perimeters_2019/FeatureServer/0/query?"

maria_params = {
    "f": "geojson",          
    "outFields": "active,datecurrent,uniquefireidentifier,fireyear,incidentname,perimeterdatetime,complexparentirwinid,state,localincidentidentifier,shape__Area,shape__Length",
    "where": "incidentname = 'Maria'",
    "returnGeometry": "true",
    "outSR": 4326,
}

GEOMAC_response = requests.get(GEOMAC_URL, params=maria_params)
GEOMAC_json = GEOMAC_response.json()

# Save perimeter updates in GeoDataFrame
perimeters_gdf = gpd.GeoDataFrame.from_features(
    GEOMAC_json["features"],
    crs="EPSG:5070"  # Required CRS for FARSITE input
)

# Save perimeters to GeoJSON
perimeters_gdf.to_file("example_outputs/maria_perimeters.geojson", driver="GeoJSON")

In [6]:
perimeters_gdf

,geometry,active,datecurrent,uniquefireidentifier,fireyear,incidentname,perimeterdatetime,complexparentirwinid,state,localincidentidentifier,shape__Area,shape__Length
0,"POLYGON ((-119.02 34.343, -119.02 34.341, -119...",N,1572825600000,2019-CAVNC-090135,2019,Maria,1572821100000,,CA,090135,6.461994e+07,50290.802512
1,"POLYGON ((-119.05 34.342, -119.05 34.342, -119...",N,1572825600000,2019-CAVNC-090135,2019,Maria,1572656400000,,CA,090135,6.201136e+07,46301.925922
2,"POLYGON ((-119.094 34.292, -119.091 34.298, -1...",N,1572566400000,2019-CAVNC-090135,2019,Maria,1572588000000,,CA,090135,7.124930e+07,35289.717123
3,"POLYGON ((-119.093 34.317, -119.09 34.319, -11...",N,1572825600000,2019-CAVNC-090135,2019,Maria,1572733320000,,CA,090135,6.460625e+07,50211.258517
4,"POLYGON ((-119.072 34.289, -119.072 34.289, -1...",N,1572825600000,2019-CAVNC-090135,2019,Maria,1572817200000,,CA,090135,5.971854e+07,66055.577037
5,"POLYGON ((-119.09 34.294, -119.09 34.294, -119...",N,1572912000000,2019-CAVNC-090135,2019,Maria,1572916500000,,CA,090135,5.946424e+07,70574.878995
